- Transformer 是现代人工智能的核心，而attention是 Transformer 最具特色的机制，在 transformer 及 attention 上花再多的时间探索都是值得。
- 从加速计算/存储的角度优化 attention 计算，都是在 attention 的计算机制上做文章：
    - vllm 中的 paged attention：推理优化
    - 以及今天要讲的 flash attention（考虑到硬件的读取和计算）：不只是推理；
        - sdpa：`torch.nn.functional.scaled_dot_product_attention`

In [18]:
from IPython.display import Image
import numpy as np
import torch


https://github.com/Dao-AILab/flash-attention

- install
    
    ```
    # pip
    pip install flash-attn --no-build-isolation
    pip install flash_attn -U --force-reinstall
    
    # source code compile
    python setup.py install
    ```
- references
    - paper
        - [FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness](https://arxiv.org/pdf/2205.14135)
        - [FlashAttention-2: Faster Attention with Better Parallelism and Work Partitioning](https://tridao.me/publications/flash2/flash2.pdf)
    - https://gordicaleksa.medium.com/eli5-flash-attention-5c44017022ad
    - https://medium.com/@e0928021388/%E7%AA%81%E7%A0%B4-transformers-%E7%9A%84%E9%80%9F%E5%BA%A6%E7%93%B6%E9%A0%B8-flash-attention-%E4%BB%8B%E7%B4%B9-28c1bc667fd9

## basics

> 算法（软件）、硬件协同优化；

- Memory
    - SRAM > HBM > DRAM
    - SRAM：Static RAM（Random Access Memory）
        - 每个 SM（Stream multiproecssors，流多处理器）192KB （A100 108个，4090 128个）
            - 108*192/1024 = 20MB
    - HBM：high bandwidth memory（4090 24GB，A100 80GB）
- GPU 读写&计算（compute-bound vs. memory-bound)
    - operations fused：将好几个 operations fuse 成一个 operation 进而减轻 memory 存取的 loading
    - 在GPU当中有非常大量的 threads （kernel） 负责执行 operation 的运算，而整个运算的过程基本上是从 HBM 当中将资料加载至 SRAM 中，执行运算并将 output 存回 HBM 当中。
    - compute-bound
        - 运算的主要时间都耗费在 operation 的计算上，HBM 的存取只占了其中一点点的时间
        - 像是多维度的矩阵相乘或是高 channel 数的 convolution 都属于这类。
    - memory-bound
        - 主要时间都耗费在 memory 的读取上，而实际的运算只占了其中一点点的时间
        - elementwise （e.g.， activation， dropout） and **reduction** （e.g.， sum， softmax， batch norm， layer norm）
- attention QKV 计算
    - 分块矩阵，然后是 loop（outer loop，inner loop，对应的是 gpu cuda 的 kernel 优化）；

### R/W and operations fused

In [89]:
Image(url='./imgs/sram_comp.png', width=500)

In [88]:
Image(url='https://miro.medium.com/v2/resize:fit:828/format:webp/0*0Yn1aLye8s6_WTOu.png', width=500)


- A **kernel** is basically a fancy way of saying “a GPU operation”.
- **Fusion** means you’re fusing/combining multiple ops together.


```
# 独立的内核调用
a = x + y  # 内核1
b = a * z  # 内核2
c = torch.relu(b)  # 内核3

# 优化后的内核（操作融合为一个内核）
# 定义操作融合的内核（使用 TorchScript）
@torch.jit.script
def fused_kernel(x, y, z):
    a = x + y
    b = a * z
    c = torch.relu(b)
    return c
```

### review of Attention

In [14]:
Image(url='./imgs/attention_steps.png', width=600)

$$
\mathbf{O} = \text{Dropout}(\text{Softmax}(\text{Mask}(\mathbf{QK}^T)))\mathbf{V}
$$

- formula
    - $(\mathbf {Q, K, V})\in\mathbb R^{N\times d}$
    - $\mathbf{QK}^T\in\mathbb R^{N\times N}$
    - $\mathbf A=\text{Dropout}(\text{Softmax}(\text{Mask}(\mathbf{QK}^T)))$
        - mask: m, softmax: sm, dropout: do
    - $\mathbf O=\mathbf {AV}\in \mathbb R^{N\times d}$
- notes
    - attention is bottlenecked by memory reads/writes
    - naive implementation requires repeated R/W from slow GPU HBM

In [90]:
Image(url='./imgs/stand_attn.png', width=800)

-  memory access 的时间复杂度为 $O（N*D + N*N）$，其中通常 N >> D（e.g.， N 为 4096 而 d 为 64），因此我们可以发现 S 和 P 的 memory access （$N*N$ 的复杂度） 便是整体 self-attention 的 bottleneck！
- 我们可以发现对于整个 self-attention 当中，其实我们真正需要的是最后面的 output O 而已，过程当中不管 P 和 S 长什么样子其实对于我们来说都没有很重要，既然他不重要为什么我们还是要将他存入 HBM 呢？ 主要是因为以下两个理由：
    - 我们需要这些 intermediate activations 来帮助我们在 backward 的时候通过 backpropagation 计算 gradients，这也使得我们很难将多个 operations fuse 成一个 operation。
    - 由于 SRAM 本身不够大，而 softmax 这种需要计算 sum 的 operation，需要整个 row 的 element 都到齐后才可以计算，使得我们沒有办法 apply 一些 divide and conquery 的 algorithm ，更使得我们没有办法把所有运算一口气在 SRAM当中计算完。

## flash attention (Tiling & Recomputation)

In [10]:
Image(url='./imgs/flash-attn.png', width=700)

In [87]:
Image(url='./imgs/stand_attn.png', width=800)

### tiling

- 这是一个现实生活的概念（谷歌搜图）
- 前向后向都会用得到
- chunking the NxN softmax/scores matrix into blocks.
- `(Q, K) => S => P => O` => `(Q, K) => O`

- softmax of vector $x\in \mathbb R^B$（element-wise）

    $$
    m(x):=\max_i x_i, \quad f(x):=[e^{x_1-m(x)}, \cdots, e^{x_B-m(x)}], \quad \ell(x)=\sum f(x), \quad \text{softmax}(x):=\frac{f(x)}{\ell(x)}
    $$
  - 减去 max value 来避免经过 exponential 后 overflow

- vectors $x^{(1)},x^{(2)}\in \mathbb R^B, x=[x^{(1)}\quad x^{(2)}]\in \mathbb R^{2B}$（element-wise）

    $$
    \begin{split}
    m(x) &= m\left( \left[ x^{(1)}, x^{(2)} \right] \right) = \max \left( m(x^{(1)}), m(x^{(2)}) \right), \\
    f(x) &= \left[ e^{m(x^{(1)}) - m(x)} f(x^{(1)}), \, e^{m(x^{(2)}) - m(x)} f(x^{(2)}) \right], \\
    \ell(x) &= \ell\left( \left[ x^{(1)}, x^{(2)} \right] \right) = e^{m(x^{(1)}) - m(x)} \ell(x^{(1)}) + e^{m(x^{(2)}) - m(x)} \ell(x^{(2)}), \\
    \text{softmax}(x) &= \frac{f(x)}{\ell(x)}.
    \end{split}
    $$
  - 这里的 $f(x^{(1)})=[e^{x_1-m(x^{(1)})}, \cdots]$

In [64]:
x = torch.tensor([1, 2, 3, 4], dtype=torch.float)
x

tensor([1., 2., 3., 4.])

In [62]:
torch.softmax(x, dim=-1)

tensor([0.0321, 0.0871, 0.2369, 0.6439])

In [63]:
m = torch.max(x)
f = torch.exp(x - m)
l = torch.sum(f)
f / l

tensor([0.0321, 0.0871, 0.2369, 0.6439])

In [67]:
x_1 = x[:2]
x_2 = x[2:]
m = torch.max(x)
m_1 = torch.max(x_1)
m_2 = torch.max(x_2)

f_1 = torch.exp(x_1 - m_1)
f_2 = torch.exp(x_2 - m_2)

l_1 = torch.sum(f_1)
l_2 = torch.sum(f_2)

f = torch.cat((torch.exp(m_1 - m) * f_1, torch.exp(m_2 - m) * f_2))
l = torch.exp(m_1 - m) * l_1 + torch.exp(m_2 - m) * l_2
f/l

tensor([0.0321, 0.0871, 0.2369, 0.6439])

In [54]:
Image(url='./imgs/flash-attn-algo.png', width=600)

- $\mathbf {Q,K,V,O}$ 分别做行分块（row blocks），$\mathbf O$ 是结果矩阵 => $4d$
    - $\mathbf Q_i,\mathbf O_i\in \mathbb R^{B_r\times d}$
    - $\mathbf {K}_j,\mathbf {V}_j\in\mathbb R^{B_c\times d}$
- $\ell$：row exp sum，$m$：row max
- 对 $T_c$（$\mathbf K_j,\mathbf V_j$） 做外循环，对 $T_r$（$\mathbf Q_i, \mathbf O_i$）做内循环
    - 内循环不断地 update $\mathbf O_i$

In [69]:
s = torch.tensor([0.1, 0.3, 0.5, 0.7])
v = torch.tensor([7, 8,  9, 10], dtype=torch.float)

In [70]:
p = torch.softmax(s, dim=-1)
p

tensor([0.1807, 0.2207, 0.2695, 0.3292])

In [71]:
p @ v

tensor(8.7472)

In [72]:
# tiling, sub-blocks
s_1, s_2 = s[:2], s[2:]
v_1, v_2 = v[:2], v[2:]

In [81]:
# step 1
p_1 = torch.softmax(s_1, dim=-1)
exp_sum_1 = torch.sum(torch.exp(s_1))
# 需要额外存储 exp_sum_1
p_1 @ v_1, exp_sum_1

(tensor(7.5498), tensor(2.4550))

In [79]:
# step 2
p_2 = torch.softmax(s_2, dim=-1)
exp_sum_2 = torch.sum(torch.exp(s_2))
p_2 @ v_2, exp_sum_2

(tensor(9.5498), tensor(3.6625))

In [83]:
# step 3
# 这里可以看到 sub-blocks 对应的计算可以独立进行（不用 care S&P 的具体形式），只需要额外存储 exp_sum
(9.5498 * 3.66 + 7.5498 * 2.455) / (2.455 + 3.66)

8.74685641864268

In [85]:
Image(url='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*i-MeAwCRNds5prU9HiSmuQ.png', width=500)

- $QKV$ 的计算发生在 SRAM 上；
- 尽管这样的方式不能让我们避免`O（N*N）` 的时间复杂度（因为我们需要 for loop 将每个 Key vector 和 Query vector 做内积，上图所示），但是这样切割成 sub-block 直接计算出结果，且不用整个 row 一起存取的方式，整个时间复杂度除以 M （sub-block 数量），同时减少许多 O（N*N） memory 存取的次数，还是可以达到非常显著的效果提升！

### Recomputation

> 不储存 intermediate activations (attn matrix, $S,P$) 而是在有需要的时候（比如 backward）再重新计算
> 
- backward only
- 概念其实类似于 gradient checkpointing（也是一个再计算的逻辑），然而 gradient checkpointing 的主要精神是稍微牺牲一些速度但是可以大幅度的减少 GPU memory 的需求 （时间换取空间的感觉），而在这边 flash attention 当中的 recomputation 这样的做法除了可以节省 GPU memory 之外还可以加速！
- 当我们在计算backward时，我们本来就要将 K， Q， and V 加载 SRAM，而与其我们在 forward 时将 S 和 P 这两个 N*N 的 matrix 存入 HBM， 并且在 backward 时再将他们两个从 HBM load 到 SRAM，我们直接用本来就在 SRAM 当中的 K， Q， and V 重新计算出 S 和 P 反而可以更快。 这点也反应了 HBM 本身相较于 SRAM 和 GPU computing 速度的差距！
- 結合了 Tiling 和 Recomputation，使得 flash attention 有办法 operations fuse 成一個 operation，更進一步避免了 HBM 的 read 和 write 的 loading，而不用担心 fusion 后使得在 backward 时会无法进行 chain rule。